### tests settings

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#let's set the parameters
M_bh = 10**6.3
nu0 = 3.3*10**(-5)

# Physical constants
G = 6.67430e-11       # m^3 kg^-1 s^-2
c = 299792458         # m/s
Msun = 1.98847e30     # kg

Rg_sun = G * Msun / c**2  # in m
#let's set the tolerance for the target frequency
tol = 0.01 * nu0  # 1% tolerance
target_min = 1e-5
target_max = 1e-4

/home/masterpops/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
# Kerr ISCO radius
def r_isco(a):
    if a==0:
        return 6
    Z1 = 1 + (1 - a**2)**(1/3) * ((1 + a)**(1/3) + (1 - a)**(1/3))
    Z2 = np.sqrt(3*a**2 + Z1**2)
    return 3 + Z2 - abs(a)/a*np.sqrt((3 - Z1)*(3 + Z1 + 2*Z2))

### PIF

#### funzione base

In [3]:
def nu_prec_solid_body(a, M, r_in, r_out, zeta=0.0):  
    _eps = 1e-14          # small tolerance

    # convert inputs to numpy arrays for safe broadcasting
    a_arr = np.asarray(a, dtype=float)
    r_in_arr = np.asarray(r_in, dtype=float)
    r_out_arr = np.asarray(r_out, dtype=float)
    zeta_arr = np.asarray(zeta, dtype=float)
    
    # mass in kg and gravitational radius Rg = G M / c^2
    Rg = Rg_sun*M
    
    # Prepare output array with broadcasting shape
    shape = np.broadcast(a_arr, r_in_arr, r_out_arr, zeta_arr).shape
    omega = np.full(shape, np.nan, dtype=float)
    
    # Quick guard: invalid radii
    invalid = (r_out_arr <= r_in_arr) | (r_in_arr <= 0)
    if np.any(invalid):
        # we'll leave those entries as NaN, but continue computing valid ones
        pass
    
    # compute common factors safely on broadcasted arrays
    # To avoid invalid broadcasting complexities, create full arrays via broadcast_to
    A = np.broadcast_to(a_arr, shape)
    RIN = np.broadcast_to(r_in_arr, shape)
    ROUT = np.broadcast_to(r_out_arr, shape)
    Z = np.broadcast_to(zeta_arr, shape)
    
    # Denominator checks
    denom_pref = 1.0 + 2.0 * Z
    denom_pref_bad = np.isclose(denom_pref, 0.0, atol=_eps)
    
    power1 = Z + 0.5
    power2 = 2.5 - Z  # 5/2 - zeta
    denom_power_term = 1.0 - (RIN / ROUT)**(power2)
    denom_power_bad = np.isclose(denom_power_term, 0.0, atol=_eps)
    
    # valid mask: rout > rin, denominators ok
    valid = (~invalid) & (~denom_pref_bad) & (~denom_power_bad)
    
    if not np.any(valid):
        omega / (2*np.pi)
    
    # compute safe quantities where valid
    # factor = (2 * a * c / Rg) * ((5 - 2 zeta) / (1 + 2 zeta))
    prefactor = (2.0 * A * c / Rg) * ((5.0 - 2.0 * Z) / denom_pref)
    
    # numerator: 1 - (r_in / r_out)^(zeta + 1/2)
    num = 1.0 - (RIN / ROUT)**(power1)
    
    # denominator: r_out^(5/2 - zeta) * r_in^(1/2 - zeta) * (1 - (r_in/r_out)^(5/2 - zeta))
    denom = (ROUT**(power2)) * (RIN**(0.5 - Z)) * denom_power_term
    
    # compute omega only for valid entries
    omega[valid] = prefactor[valid] * (num[valid] / denom[valid])
    
    # a == 0 => omega = 0 (ensured by prefactor). if tiny numerical noise appears, force exact zero:
    omega[np.isclose(A, 0.0, atol=_eps) & valid] = 0.0
    nu = omega / (2.0 * np.pi)

    return nu


#### funzioni brutte

In [ ]:
a_range = np.linspace(-0.999, 0.999, 10001)  # Spins
r_ins = np.array([np.exp(np.linspace(np.log(r_isco(a)), np.log(5e2), 400)) for a in a_range])
#r_outs = [np.exp(np.linspace(np.log(r_isco(a)), np.log(1e4), 800)) for a in a_range]

# broadcast zeta
zetas = np.array([-0.45, 0, 0.5])
ZETA = zetas[:, None, None, None]  # shape (Nz,1,1,1)

In [ ]:
from scipy.optimize import root_scalar
def f_rout(r_out, a, M, r_in, zeta):
    return nu_prec_solid_body(a, M, r_in, r_out, zeta) - nu0

matches = []
for i, a in enumerate(a_range):
    for zeta in zetas:
        for r_in in r_ins[i]:
            # bracket for r_out (must be > r_in)
            rlow = r_in * 1.05
            rhigh = 1000

            try:
                sol = root_scalar(
                    f_rout, args=(a, M_bh, r_in, zeta),
                    bracket=[rlow, rhigh], method="brentq"
                )
                if sol.converged:
                    r_out = sol.root
                    freq = nu_prec_solid_body(a, M_bh, r_in, r_out, zeta)
                    matches.append((a, r_in, r_out, zeta, freq))
            except ValueError:
                pass

In [ ]:
matches = []
for i, a in enumerate(a_range):
    for r_in in r_ins[i]:
        r_out_grid = np.exp(np.linspace(np.log(r_in*1.05), np.log(1e3), 800))
        
        for zeta in zetas:
            for r_out in r_out_grid:
                freq = nu_prec_solid_body(a, M_bh, r_in, r_out, zeta)
                if abs(freq - nu0) < tol:
                    matches.append((a, r_in, r_out, zeta, freq))

if matches.size > 0:
    print(f"\nFOUND MATCHES:")
    for m in matches:
        print(f"a={m[0]}, r_in={m[1]} R_g, r_out={m[2]} R_g, zeta={m[3]} → ν={m[4]} Hz")
